In [ ]:
import numpy as np
import scipy as sp

from plotter import Plotter

In [ ]:
write = True

In [ ]:
study = "MSAT"
file_path = "../data/TX2008.V1.P010.dat"
data = np.loadtxt(file_path)
n_material = np.size(data,0)
lons = data[:n_material,1]
lats = data[:n_material,0]
cijs = data[:n_material,3:24]

In [ ]:
betas = np.load(f"Data/{study}/betas_{study}.npy")
thetas = np.load(f"Data/{study}/thetas_{study}.npy")
sigmas = np.load(f"Data/{study}/sigmas_{study}.npy")
phis = np.load(f"Data/{study}/phis_{study}.npy")
dp_m1 = sp.io.loadmat(f"Data/{study}/decomp_percentages_{study}.mat")["DP_MS_axes"]
dp_m2 = sp.io.loadmat(f"Data/{study}/decomp_percentages_{study}.mat")["DP_MS_axes_X3_stiff"]
dp_m3 = sp.io.loadmat(f"Data/{study}/decomp_percentages_{study}.mat")["DP_I"]
dp_m1_bug = sp.io.loadmat(f"Data/{study}/decomp_percentages_{study}_bug.mat")["DP_MS_axes"]
ani_i = sp.io.loadmat("Data/MSAT/anisotropy_index_MSAT.mat")['C']

In [ ]:
def cumulation(decomp_percentages):
    n = decomp_percentages.shape[1]
    cumulative_percentages = np.copy(decomp_percentages)
    for i in range(1,n):
        cumulative_percentages[:,i] += cumulative_percentages[:,i-1]
    return cumulative_percentages

def get_alpha(p_cumulative):
    return np.rad2deg(np.arcsin(1 - p_cumulative)) 
        
cumulative_dp_m1 = cumulation(dp_m1)
cumulative_dp_m2 = cumulation(dp_m2)
cumulative_dp_m3 = cumulation(dp_m3)

alphas_m1 = np.zeros(np.shape(dp_m1))
alphas_m2 = np.zeros(np.shape(dp_m2))
alphas_m3 = np.zeros(np.shape(dp_m3))
for i in range(dp_m1.shape[1]):
    alphas_m1[:,i] = get_alpha(cumulative_dp_m1[:,i])
    alphas_m2[:,i] = get_alpha(cumulative_dp_m2[:,i])
    alphas_m3[:,i] = get_alpha(cumulative_dp_m3[:,i])

In [ ]:
symmetry_classes = ["ISO","XISO","TET","ORTH","MONO"]

In [ ]:
# alpha vs beta cmaps

cmaps = [("RdYlGn", False,  0.0,  4.0),
         (   "jet", False,  0.0,  2.0),
         (  "BrBG", False,  0.0,  1.5),
         (  "PiYG", False,  0.0,  2.0),
         (  "PRGn", False,  0.0,  1.0)]

rcmaps = [("seismic", False, None, None),
          (   "Reds", False,  0.0,  0.5),
          (   "Reds", False,  0.0,  0.3),
          (   "Reds", False,  0.0,  0.5),
          (   "Reds", False,  0.0,  0.5)]

In [ ]:
plot = Plotter(lons, lats, central_longitude=0, write=write)

In [ ]:
# alpha vs beta

for i, sigma in enumerate(symmetry_classes):
    x = betas[i,:]
    y = alphas_m1[:,i]
    xlabel = fr"$\beta_{{{sigma}}}$ (°)"    
    ylabel = fr"$\alpha_{{{sigma}}}$ (°)"   
    filename = f"alpha_beta_{sigma}_{study}"
    plot.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmaps[i], residual=True, rcmaps=rcmaps[i], trend=True, aspect="equal", filename=filename)

In [ ]:
for i, sigma in enumerate(["XISO"], start=1):

    sigma2 = sigma
    if sigma == "XISO":
        sigma2 = "HEX"

    # plot for betas
    parameter = betas[i,:]
    title = fr"$\beta_{{{sigma}}}$ (°)"
    cmap = ["jet", False, 0.00, 2.00]
    filename = f"beta_{sigma}_{study}"
    plot.global_plot(parameter, title, cmaps=cmap, filename=filename)
    
    # plot for thetas
    parameter = abs(np.cos(thetas[i,:]))
    title = "|cos (azimuthal angle)|"
    cmap = ["jet", True, 0.00, 1.00]
    filename = f"azimuthal_angle_{sigma}_{study}"
    plot.global_plot(parameter, title, cmaps=cmap, filename=filename)
    
    # plot for thetas with points corresponding to sub-vertical phis masked    
    parameter = abs(np.cos(thetas[i,:]))
    parameter[np.cos(2*phis[i,:])>0.5] = np.nan
    title = "|cos (azimuthal angle)|"
    cmap = ["jet", True, 0.00, 1.00]
    filename = f"azimuthal_angle_subverticals_masked_{sigma}_{study}"
    plot.global_plot(parameter, title, cmaps=cmap, filename=filename)

    # plot for sigmas
    parameter = np.rad2deg(sigmas[i,:])
    title = r"$\sigma$ (°)"
    cmap = ["jet", True, None, None]
    filename = f"sigma_{sigma}_{study}"
    plot.global_plot(parameter, title, cmaps=cmap, filename=filename)
    
    # plot for phis
    parameter = np.cos(2*phis[i,:])
    title = f"cos (2 * polar angle)"
    cmap = ["jet", True, -1.00, 1.00]
    filename = f"polar_angle_{sigma}_{study}"
    plot.global_plot(parameter, title, cmaps=cmap, filename=filename)
    
    # plot for proportion of hexagonality as in the publication (bug)
    # formula valid only when sigma = XISO
    parameter = dp_m1_bug[:,i]/np.sum(dp_m1_bug[:,i:], axis=1)
    title = f"proportion_of_{sigma2}_publication"
    cmap = ["jet", True, 0.00, 1.00]
    filename = f"{title}_{study}"
    plot.global_plot(parameter, title, cmaps=cmap, filename=filename)
    
    # plot for proportion of hexagonality as intended in the publication (bug fixed)
    # formula valid only when sigma = XISO
    parameter = dp_m1[:,i]/np.sum(dp_m1[:,i:], axis=1)
    title = f"proportion_of_{sigma2}"
    cmap = ["jet", True, 0.00, 1.00]
    filename = f"{title}_{study}"
    plot.global_plot(parameter, title, cmaps=cmap, filename=filename)

    # plot for hexagonality percentage
    parameter = dp_m1[:,i] * 100
    title = f"{sigma2}_percentage"
    cmap = ["jet", True, 0.00, 4.00]
    filename = f"{title}_{study}"
    plot.global_plot(parameter, title, cmaps=cmap, filename=filename)

In [ ]:
# XISO pairings

# alpha_XISO
a = alphas_m1[:,1]
a_label = r"$\alpha_{XISO}$ (°)"
a_cmap = cmaps[1] 

# beta_XISO
b = betas[1,:]
b_label = r"$\beta_{XISO}$ (°)"
b_cmap = cmaps[1]

# HEX_percentage
c = dp_m1[:,1] * 100
c_label = "HEX_percentage"
c_cmap = ["jet", True, 0.00, 4.00]

# proportion_of_HEX
d = dp_m1[:,1]/np.sum(dp_m1[:,1:], axis=1)
d_label = "proportion_of_HEX"
d_cmap = ["jet", True, 0.00, 1.00]

filename = f"alpha_beta_XISO_{study}"
plot.correlation_plots(b, a, b_label, a_label, cmaps1=b_cmap, cmaps2=a_cmap, residual=False, trend=True, aspect="equal", filename=filename)

filename = f"alpha_XISO_HEX_percentage_{study}"
plot.correlation_plots(a, c, a_label, c_label, cmaps1=a_cmap, cmaps2=c_cmap, residual=False, filename=filename)

filename = f"alpha_XISO_proportion_of_HEX_{study}"
plot.correlation_plots(a, d, a_label, d_label, cmaps1=a_cmap, cmaps2=d_cmap, residual=False, filename=filename)

filename = f"beta_XISO_HEX_percentage_{study}"
plot.correlation_plots(b, c, b_label, c_label, cmaps1=b_cmap, cmaps2=c_cmap, residual=False, filename=filename)

filename = f"beta_XISO_proportion_of_HEX_{study}"
plot.correlation_plots(b, d, b_label, d_label, cmaps1=b_cmap, cmaps2=d_cmap, residual=False, filename=filename)

filename = f"HEX_percentage_proportion_of_HEX_{study}"
plot.correlation_plots(c, d, c_label, d_label, cmaps1=c_cmap, cmaps2=d_cmap, residual=False, filename=filename)

In [ ]:
for i, sigma in enumerate(["ISO"], start=0):

    # anisotropy index vs beta
    x = betas[i,:]
    y = ani_i[0,:]
    xlabel = fr"$\beta_{{{sigma}}}$ (°)"    
    ylabel = "universal_elastic_anisotropy_index"
    cmap1 = ["jet", False, 0.00, 6.00]
    cmap2 = ["jet", False, 0.00, 0.10]
    filename = f"universal_elastic_anisotropy_index_beta_{sigma}_{study}"
    plot.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmap1, cmaps2=cmap2, residual=False, filename=filename)

In [ ]:
# plot for anisotropy index
parameter = ani_i[0,:]
title = "universal_elastic_anisotropy_index"
cmap = ["jet", True, 0.00, 0.10]
filename = f"{title}_{study}"
plot.global_plot(parameter, title, cmaps=cmap, filename=filename)